# **importing necessary libraries**

In [35]:
pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Fou

In [31]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig, AutoTokenizer, TFAutoModel
import numpy as np
import pandas as pd
import evaluate
import torch
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from datasets import Dataset


# **Preparing data**

Firstly I read the comments dataset and preprocess it.


I decided to have three classes :


*   پیشنهاد می‌کنم
*   پیشنهاد نمی‌کنم
*   مطمئن نیستم


which considered as posetive, negetive and natural.
then we drop comments with less than 2 words so that the dataset wouldn't be Heterogeneous.


In [1]:
import pandas as pd
colnames=['comments','labels']
digi_df = pd.read_csv('./diji_comments_scraped.csv', names=colnames ,skiprows=1,index_col=None )

FileNotFoundError: [Errno 2] No such file or directory: './diji_comments_scraped.csv'

In [33]:
digi_df.head()

,comments,labels
0,در کل خوبه,پیشنهاد می‌کنم
1,فقط میتونم بگم عالیه,پیشنهاد می‌کنم
2,خیلی خوبه مخصوصا برای محل کار,پیشنهاد می‌کنم
3,خوبه کار راه اندازه ولی حیف که چرخش نداره,پیشنهاد می‌کنم
4,نسبت به قیمتش خیلی محصول خوب و کاربردیه,پیشنهاد می‌کنم


In [34]:
len(digi_df['labels'])

1919

In [35]:
non_valid_labels = []
for i, label in enumerate(digi_df['labels']):
  if label == 'پیشنهاد می‌کنم' or label == 'مطمئن نیستم' or label == 'پیشنهاد نمی‌کنم':
    pass
  else:
    non_valid_labels.append(i)


In [36]:
digi_df = digi_df.drop(non_valid_labels)

In [37]:
digi_df['labels'].value_counts()

,count
labels,
پیشنهاد می‌کنم,1695
مطمئن نیستم,109
پیشنهاد نمی‌کنم,92


In [38]:
count = np.zeros(len(digi_df['comments']))
for i,comment in enumerate(digi_df['comments']):
  count[i] =len(comment.split())

In [39]:
digi_df = digi_df[count>1].reset_index()

*From 1839 comments, We end up with 1378 heterogeneous ones.*

In [41]:
def label_digi(x):
  if x['labels']  == 'پیشنهاد می‌کنم' :
    return 0
  elif  x['labels']  == 'پیشنهاد نمی‌کنم':
    return 1
  else:
    return 2

In [42]:
digi_df['labels'] = digi_df.apply(lambda x: label_digi(x), axis=1)
digi_df.head()

,index,comments,labels
0,0,در کل خوبه,0
1,1,فقط میتونم بگم عالیه,0
2,2,خیلی خوبه مخصوصا برای محل کار,0
3,3,خوبه کار راه اندازه ولی حیف که چرخش نداره,0
4,4,نسبت به قیمتش خیلی محصول خوب و کاربردیه,0


I converted the labels to 0 , 1 and 2 so that the classification would be easier to the model.

# **Defining the model , tokenizer and triner from hugging face:**

In [44]:
train_texts, val_texts, train_labels, val_labels = train_test_split(digi_df['comments'], digi_df['labels'], test_size=.2)

In [45]:
train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels})

In [66]:
config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = AutoModelForSequenceClassification.from_pretrained("HooshvareLab/bert-base-parsbert-uncased", num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
training_args = TrainingArguments(output_dir="test_trainer")

In [60]:
metric = evaluate.load("accuracy")

In [61]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [62]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [63]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True,max_length=32)


train_tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
val_tokenized_datasets = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1157 [00:00<?, ? examples/s]

Map:   0%|          | 0/290 [00:00<?, ? examples/s]

In [64]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=val_tokenized_datasets,
    compute_metrics=compute_metrics,
    # data_collator=data_collator
)

In [68]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.474780,0.820690
2,No log,0.533914,0.858621
3,No log,0.527112,0.851724


TrainOutput(global_step=435, training_loss=0.29912800405217316, metrics={'train_runtime': 62.0918, 'train_samples_per_second': 55.901, 'train_steps_per_second': 7.006, 'total_flos': 57079167058368.0, 'train_loss': 0.29912800405217316, 'epoch': 3.0})

In [67]:
trainer.evaluate()



{'eval_loss': 0.46845924854278564,
 'eval_accuracy': 0.8448275862068966,
 'eval_runtime': 0.7091,
 'eval_samples_per_second': 408.979,
 'eval_steps_per_second': 52.18,
 'epoch': 3.0}

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/651M [00:00<?, ?B/s]

events.out.tfevents.1722780699.635160a05d01.13463.4:   0%|          | 0.00/4.95k [00:00<?, ?B/s]

events.out.tfevents.1722784685.635160a05d01.21589.0:   0%|          | 0.00/4.95k [00:00<?, ?B/s]

events.out.tfevents.1722780578.635160a05d01.13463.3:   0%|          | 0.00/4.80k [00:00<?, ?B/s]

Upload 17 LFS files:   0%|          | 0/17 [00:00<?, ?it/s]

...

events.out.tfevents.1722786019.635160a05d01.21589.10:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

events.out.tfevents.1722786058.635160a05d01.21589.11:   0%|          | 0.00/6.28k [00:00<?, ?B/s]

events.out.tfevents.1722789077.635160a05d01.21589.12:   0%|          | 0.00/411 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kimik76/test_trainer/commit/662f0c2a864c013dbdd5bf911fe85c95bb2dd23e', commit_message='End of training', commit_description='', oid='662f0c2a864c013dbdd5bf911fe85c95bb2dd23e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
text = "ما در هوشواره معتقدیم با انتقال صحیح دانش و آگاهی، همه افراد می‌توانندت."
a=tokenizer(text, return_tensors="tf")
a

{'input_ids': <tf.Tensor: shape=(1, 19), dtype=int32, numpy=
array([[    2,  2179,  2028,  3637,  2963, 10517,  2037,  3485,  5371,
         2338,   331,  4801,   300,  2440,  2698,  3247,  1174,    15,
            4]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 19), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 19), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
      dtype=int32)>}

In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
RESPOSES = {
    0 : "پیشنهاد میکنم",
    1 : "پیشنهاد نمیکنم",
    2 : "مطمئن نیستم"
}

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

# Load the tokenizer and the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model2 = TFAutoModelForSequenceClassification.from_pretrained("kimik76/test_trainer")

# Prepare the inputs
inputs = tokenizer("وای چه پنکه خنک کننده ای", return_tensors="tf")

# Get logits from the model
logits = model2(**inputs).logits

# Get the predicted class (assuming this is a classification task)
predicted_class = tf.argmax(logits, axis=-1).numpy()

print(f"نظر تحلیلی مدل: {RESPOSES[int(predicted_class)]}")


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


نظر تحلیلی مدل: پیشنهاد میکنم


In [ ]:
# Prepare the inputs
inputs = tokenizer("حیف پول", return_tensors="tf")

# Get logits from the model
logits = model2(**inputs).logits
# Get the predicted class (assuming this is a classification task)
predicted_class = tf.argmax(logits, axis=-1).numpy()

print(f"نظر تحلیلی مدل: {RESPOSES[int(predicted_class)]}")


نظر تحلیلی مدل: پیشنهاد نمیکنم


In [ ]:
# Prepare the inputs
inputs = tokenizer("هنوز مطمئن نیستم ", return_tensors="tf")

# Get logits from the model
logits = model2(**inputs).logits
# Get the predicted class (assuming this is a classification task)
predicted_class = tf.argmax(logits, axis=-1).numpy()

print(f"نظر تحلیلی مدل: {RESPOSES[int(predicted_class)]}")


نظر تحلیلی مدل: مطمئن نیستم


In [ ]:
!pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.2 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

import streamlit as st

st.write('Hello, *World!* :sunglasses:')

Overwriting app.py


In [ ]:
!npm install localtunnel


up to date, audited 60 packages in 848ms

5 packages are looking for funding
  run `npm fund` for details

6 vulnerabilities (1 low, 3 moderate, 2 high)

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
!streamlit run app.py &>/content/logs.txt &


In [ ]:
!npx localtunnel --port 8501


your url is: https://slick-sites-repair.loca.lt
^C


In [ ]:
import streamlit as st
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

# Load the tokenizer and the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model2 = TFAutoModelForSequenceClassification.from_pretrained("kimik76/test_trainer")

# Dictionary of responses
RESPONSES = ["Class 0", "Class 1", "Class 2"]  # Update with actual class names if available

# Streamlit app
st.title("Text Classification with ParsBERT")

# Input text box
user_input = st.text_input("Enter text to classify:")

if user_input:
    # Prepare the inputs
    inputs = tokenizer(user_input, return_tensors="tf")

    # Get logits from the model
    logits = model2(**inputs).logits

    # Get the predicted class
    predicted_class = tf.argmax(logits, axis=-1).numpy()[0]

    # Display the result
    st.write(f"Model's analytical opinion: {RESPONSES[int(predicted_class)]}")
